In [3]:
import pandas as pd
import json
from dotenv import load_dotenv
from src.algorithm import improve_system

load_dotenv()

True

#### At this point, we've run some examples through our LLM graph. If not, refer to this [post](https://langchain-ai.github.io/langgraph/tutorials/introduction/) to get started.

#### We have a graph, let's save it first.

In [32]:
with open("example-data/graph-healthcare.json", "r") as json_file:
    graph_json = json.load(json_file)

edges = graph_json["edges"]

In [33]:
from src.middleware import convert_edges_to_dag

dag = convert_edges_to_dag(edges=edges)

#### Read in dataset with your prompts and LLM responses

In [34]:
llm_responses = pd.read_excel("example-data/healthcare-responses.xlsx")
llm_responses = llm_responses.tail(10)
llm_responses

,document,batch_id,prompt-1,prompt-2,prompt-3,prompt-4,prompt-5,prompt-6,prompt-7,response-1,...,response-6,response-7,remove_pii_1,remove_pii_2,remove_pii_3,remove_pii_4,remove_pii_5,"Symptoms, Reason to call, Rec",Summarized,Notes
199,<s>[INST] Classify the following call transcri...,4d7d73f4543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...","Here is the input with the patient's name, pho...",...,"{\n ""main_reason"": ""To understand the break...",The provided input suggests that the main reas...,1,1.0,1.0,1.0,1.0,1,1,NaN
200,<s>[INST] Classify the following call transcri...,5a499d6a543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...",Here is the redacted transcript:\n\n<s>[INST] ...,...,Here is the dictionary with the required infor...,The main reason for the request is to obtain m...,1,1.0,1.0,1.0,1.0,1,1,NaN
201,<s>[INST] Classify the following call transcri...,67a59a36543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...",Here is the redacted transcript:\n\n<s>[INST] ...,...,Here is the dictionary with the requested info...,The provided input is a dictionary containing ...,0,1.0,1.0,1.0,1.0,1,1,NaN
202,<s>[INST] Classify the following call transcri...,740d80f4543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...","Here is the input with the patient's name, pho...",...,"{\n ""main_reason"": [""Requesting medical rec...",The main reason for this request is to obtain ...,1,1.0,1.0,1.0,1.0,1,1,NaN
203,<s>[INST] Classify the following call transcri...,823cd472543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...","Here is the input with the patient's name, pho...",...,Here is the dictionary with the requested info...,Here is a general summary of the provided info...,1,1.0,1.0,1.0,1.0,1,1,NaN
204,<s>[INST] Classify the following call transcri...,8dc53726543711ef907f86d6773e4b0f,"Your task is to redact the patient's name, pho...",Your task is to redact the patient's ID from t...,Your task is to redact the patient's birthdate...,Your task is to redact the patient's medicatio...,Your task is to redact the patient's policy nu...,\n You have a task to return a dictionary w...,"\n Given the input, create a general summar...","Here is the input with the patient's name, pho...",...,Here is the dictionary with the requested info...,The input provided suggests that the individua...,1,1.0,1.0,1.0,1.0,1,1,NaN
205,<s>[INST] Classify the following call transcri...,99fcf84e543

#### Use LLM-as-a-Judge to run the responses for validation

In [38]:
from src.llm_as_judge import run_validation_using_LLM

# node_input_output_mappings = {
#     "extract_keywords_from_title": ("prompt-1", "output-keywords"),
#     "tool_search": ("prompt-2", "output-article_search"),
#     "summarize_tone_sentiment_of_replies": ("prompt-3", "output-replies"),
#     "generate_reply": ( "prompt-4", "output-reply")
# }

node_input_output_mappings = {
    "pii_name_number_email": ("prompt-1", "response-1"),
    "pii_id": ("prompt-1", "response-2"),
    "pii_birthdate": ("prompt-1", "response-3"),
    "pii_medications": ("prompt-1", "response-4"),
    "pii_insurance": ("prompt-1", "response-5"),
    "extractor": ("prompt-1", "response-6"),
    "summarizer": ("prompt-1", "response-7"),
}

llm_response_df = run_validation_using_LLM(
    dataframe=llm_responses, node_input_output_mappings=node_input_output_mappings
)

In [39]:
# from scipe import convert_to_dag, run_validation_using_llm, find_problematic_node
# # "Systematic Chain Improvement and Problem Evaluation"

# # We have the graph from langgraph
# graph = Langgraph()
# # Middleware to convert it to a format that we need
# # People can write this however
# dag = convert_to_dag(graph) # We'll need this later

# data_of_inputs_outputs = pd.DataFrame()

# llm_responses_df = run_validation_using_llm(data_of_inputs_outputs)

# node = find_problematic_node(llm_responses_df)

# print(node)

# "Node: 1 is problematic"
# "It has 33% chance of failing on it's own, compared to 9% chance of failing"
# "because of node(n-1). Please focus on fixing Node 1."

# ## What are some ways node 1 can be fixed.
# ## Could we analyze it?


In [41]:
improve_system(data=llm_response_df, dag=dag)

Analyzing node: summarizer
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(summarizer fails | extractor fails): 0.0000

Analyzing node: extractor
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(extractor fails | pii_insurance fails): 0.0000

Analyzing node: pii_insurance
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure probabilities given upstream dependency failures:
  P(pii_insurance fails | pii_medications fails): 0.0000

Analyzing node: pii_medications
Overall failure probability for this node: 0.0000
Independent failure probability: 0.0000
Node failure because dep fails: 0
Conditional failure 

/Users/ankushgarg/Desktop/MIDS/epic-data-lab/scipe/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/scipe/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/scipe/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/scipe/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/scipe/src/algorithm.py:23: RuntimeWarning: invalid value encountered in scalar divide
  result = (node_fails & dep_fails).sum() / dep_fails.sum()
/Users/ankushgarg/Desktop/MIDS/epic-data-lab/

ValueError: max() arg is an empty sequence

In [ ]:
llm_response_df